# Using Covalent with PennyLane

[PennyLane](https://pennylane.ai/) is a popular Python library for differentiable programming of quantum computers that is well suited for quantum machine learning tasks. In this tutorial, we will demonstrate how to integrate Covalent with PennyLane for a simple hybrid quantum-classical optimization task. This is based on the PennyLane demo: [Plugins and Hybrid Computation](https://pennylane.ai/qml/demos/tutorial_plugins_hybrid.html).

In essence, we will build a simple photonic circuit with two qumodes (i.e., photonic analog of qubits, which are referred to as "wires" in PennyLane) using PennyLane's Strawberry Fields plugin. The circuit is initialized with the state $\ket{1,0}$ and contains a beamsplitter with two free parameters $\theta$ and $\phi$, which together determine the transmission and reflection amplitudes. The goal is to optimize the beamsplitter parameters $(\theta, \phi)$ such that the expectation value of the photon number in the second wire is maximized. In a realistic hybrid implementation, the expected photon number would be measured with a photonic quantum computer, whereas the optimization would be done on a classical computer. We will see how to use Covalent to manage such hybrid workflows. We refer the reader to the original PennyLane demo for more details on the photonic circuit.

In addition to Covalent, this tutorial uses PennyLane as well as its Strawberry Fields plugin. So we first do the following installations.

In [ ]:
# !pip install cova
# !pip install pennylane
# !pip install pennylane-sf

Then we start the Covalent server.

In [ ]:
# !covalent start

Finally, let us import the necessary libraries.

In [2]:
import pennylane as qml
from pennylane import numpy as np
import covalent as ct

## Construct the workflow

In [2]:
dev_qubit = qml.device('default.qubit', wires=1)
dev_fock = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)

In [3]:
@ct.electron
@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

@ct.electron
@qml.qnode(dev_fock)
def photon_redirection(params):
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0,1])
    return qml.expval(qml.NumberOperator(1))

@ct.electron
def cost(params, phi1=0.5, phi2=0.1):
    qubit_result = qubit_rotation(phi1, phi2)
    photon_result = photon_redirection(params)
    return np.abs(qubit_result - photon_result)**2

@ct.electron
def get_optimizer():
    return qml.GradientDescentOptimizer(stepsize=0.4)

@ct.electron
def get_initial_params(init_params):
    return np.array(init_params, requires_grad=True)

@ct.electron
def training(opt, init_params, cost, steps):
    params = init_params
    for _ in range(steps):
        params = opt.step(cost, params)
    return params, cost(params)

@ct.lattice
def workflow(init_params, steps):
    opt = get_optimizer()
    params = get_initial_params(init_params)
    params, final_cost = training(opt, params, cost, steps)
    return params, final_cost

In [20]:
result, final_cost = workflow([0.01, 0.01], 50);
print(photon_redirection(result))
print(qubit_rotation(0.5, 0.1))

0.8731982995880654
0.8731983044562817


In [4]:
dispatch_id = ct.dispatch(workflow)([0.01, 0.01], 50)

In [5]:
dispatch_id

'f447e032-6a52-4b4a-a1bd-7c88b8591a9f'

In [10]:
result = ct.get_result(dispatch_id=dispatch_id, results_dir="./results")
print(result)


Lattice Result
status: COMPLETED
result: (tensor([1.20671364, 0.01      ], requires_grad=True), tensor(2.36995297e-17, requires_grad=True))
inputs: {'args': [[0.01, 0.01], 50], 'kwargs': {}}
error: None

start_time: 2022-06-01 19:38:27.115063+00:00
end_time: 2022-06-01 19:38:34.457795+00:00

results_dir: /Users/ruihaoli/Library/CloudStorage/OneDrive-TheUniversityofSydney(Students)/CWRU 2022/Agnostiq Internship/Progress/Covalent_PennyLane/results
dispatch_id: f447e032-6a52-4b4a-a1bd-7c88b8591a9f

Node Outputs
------------
get_optimizer(0): <pennylane.optimize.gradient_descent.GradientDescentOptimizer object at 0x7ff1113e43a0>
get_initial_params(1): [0.01 0.01]
:electron_list:(2): [0.01, 0.01]
:parameter:0.01(3): 0.01
:parameter:0.01(4): 0.01
training(5): (tensor([1.20671364, 0.01      ], requires_grad=True), tensor(2.36995297e-17, requires_grad=True))
:parameter:<function cost at 0x7ff11071e310>(6): <function cost at 0x7ff1113ea310>
:parameter:50(7): 50
:generated:training()[0](8): [1.

In [11]:
print(photon_redirection([1.20671364, 0.01]))
print(qubit_rotation(0.5, 0.1))

0.8731983021146449
0.8731983044562817
